## Introduction

Using this notebook, you can see the proportion of elderly people who visited the hospital within 7 days after a dust event in Ahwaz district.
tst

In [1]:
import geopandas as gpd, folium, branca
from folium.plugins import Search

In [2]:
df4 = gpd.read_file(r"D:\notebooks\data\geo_all_eldery.geojson", driver='GeoJSON')

In [3]:
df4.head()

,neighbour,all_eldery_percent,geometry
0,20metri shahrdari,2.923977,"MULTIPOLYGON (((283106.83944 3468138.54001, 28..."
1,30metri,3.718200,"MULTIPOLYGON (((279668.63528 3467103.99975, 27..."
2,400 dastgah,6.818182,"MULTIPOLYGON (((284070.46376 3468173.36741, 28..."
3,aameri,5.263158,"MULTIPOLYGON (((280904.17879 3469157.84934, 28..."
4,ahan afshar,5.138340,"MULTIPOLYGON (((276267.08844 3469009.38253, 27..."


In [4]:
df4_sorted = df4.sort_values(by='all_eldery_percent', ascending=False)

df4_sorted.head(5).append(df4_sorted.tail(5))[['neighbour','all_eldery_percent']]

C:\Users\Mahsa\AppData\Local\Temp\ipykernel_252\4100622557.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df4_sorted.head(5).append(df4_sorted.tail(5))[['neighbour','all_eldery_percent']]


,neighbour,all_eldery_percent
18,boustan,44.444444
100,shahrak elahie,36.363636
84,pardis3,16.666667
52,kouy polis,15.789474
45,kian shahr,14.503817
96,shahed,1.098901
62,kouye silou,0.000000
80,nezam mohandesi,0.000000
6,al-e-saafi,0.000000
50,kouy asatid,0.000000


In [5]:
# Neighbourhoods with the most hospital visits
df4_sorted.head(10)

,neighbour,all_eldery_percent,geometry
18,boustan,44.444444,"MULTIPOLYGON (((275211.82449 3463748.40561, 27..."
100,shahrak elahie,36.363636,"MULTIPOLYGON (((275382.48108 3462982.43533, 27..."
84,pardis3,16.666667,"MULTIPOLYGON (((270091.14039 3462407.75438, 27..."
52,kouy polis,15.789474,"MULTIPOLYGON (((283920.38870 3465504.53047, 28..."
45,kian shahr,14.503817,"MULTIPOLYGON (((277349.33931 3473174.51569, 27..."
95,seyed khalf,13.836478,"MULTIPOLYGON (((279684.29189 3471673.00227, 27..."
57,kouye mahdis,13.513514,"MULTIPOLYGON (((282392.43138 3463138.84535, 28..."
16,bazar abdolhamid,12.500000,"MULTIPOLYGON (((279782.93551 3467498.49429, 27..."
35,kaantex,12.274368,"MULTIPOLYGON (((278740.01965 3460339.87092, 27..."
28,faz5 padad shahr,11.428571,"MULTIPOLYGON (((281625.13818 3463223.51219, 28..."


In [5]:
min, max = df4['all_eldery_percent'].quantile([0.05,0.95]).apply(lambda x: round(x, 2))

mean = round(df4['all_eldery_percent'].mean(),2)


print(f"Min: {min}", f"Max: {max}", f"Mean: {mean}", sep="\n\n")

Min: 1.24

Max: 13.55

Mean: 6.09


In [6]:
colormap = branca.colormap.LinearColormap(
    colors=['#f2f0f7','#cbc9e2','#9e9ac8','#756bb1','#54278f'],
    index=df4['all_eldery_percent'].quantile([0.2,0.4,0.6,0.8]),
    vmin=min,
    vmax=max
)

colormap.caption="Percentage of dust-related hospital visits, 1-6 days after the event "

colormap

In [7]:
# Instantiate our folium Map.
m = folium.Map(location=[31.33, 48.68], zoom_start=12)

# Create some folium GeoJson objects from our GeoDataFrames.
neighbourgeo = folium.GeoJson(df4,
                          name='> 65',
                          style_function=lambda x: {'fillColor': colormap(x['properties']['all_eldery_percent']), 'color': 'black',
                                                    'weight':2, 'fillOpacity':0.6},
                          tooltip=folium.GeoJsonTooltip(fields=['neighbour', 'all_eldery_percent'], 
                                            aliases=['Neighbourhood', 'Percentage'], 
                                            localize=True)
                         ).add_to(m)


statesearch = Search(layer=neighbourgeo, 
                     geom_type='Polygon', 
                     placeholder="Search for a Ahwaz Neighbourhood", 
                     collapsed=False, 
                     search_label='neighbour',
                     weight=3
                    ).add_to(m)

folium.LayerControl().add_to(m)

# And the Color Map legend.
colormap.add_to(m)

m